In [ ]:
import numpy as np
import qutip as qt
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.rcParams.update({
    "savefig.facecolor": (1.0, 1.0, 1.0, 1),  # blue  with alpha = 20%
})

## Hamiltonian

$$\hat{H} = \hat{H}_{b1} + \hat{H}_{b2} + \hat{H}_I$$

### Transmon Bare Hamilonian

$ \hat{H}_{b1} = \sum_j \omega_j | j \rangle\langle j | $ where $\omega_{j}=\left(\omega-\frac{\eta}{2}\right) j+\frac{\eta}{2} j^{2}$ which is also 

$$\hat{H}_{b1} = \omega \hat{c}^{\dagger} \hat{c}+\frac{\eta}{2} \hat{c}^{\dagger} \hat{c}\left(\hat{c}^{\dagger} \hat{c}-1\right)$$

### Resonator Bare Hamiltonian

$$\hat{H}_{b2} = \left(\omega - \Delta \right) \hat{a}^{\dagger}\hat{a}$$

### The RWA interaction Hamiltonian

The coupling constant is given as $g_{k, k+1} \approx g \sqrt{k+1}\left(1+\frac{\eta}{2 \omega} k\right)$. Thus this can be written as

$$g_{k, k+1} \approx g \sqrt{\hat{c}^{\dagger}\hat{c}+1}\left(1+\frac{\eta}{2 \omega} \hat{c}^{\dagger}\hat{c}\right)$$

Therefore the RWA interaction Hamiltonian is as follows

$$\hat{H}_I = \sum_{k, n} g_{k, k+1} \sqrt{n}|k+1, n-1\rangle\langle k, n|+\text { H.c. }$$

$$\hat{H}_I =  g \left(1+\frac{\eta}{2 \omega} \hat{c}^{\dagger}\hat{c}\right) \left(\hat{a}^{\dagger}\hat{c} + \hat{a}\hat{c}^{\dagger}\right)$$

In [ ]:
# Function that outputs energy for bare Hamiltonian of the transmons
def H_q(state, eta, w01):
    return(w01*state + (eta/2)*state*(state - 1))

# Function that outputs energy for bare Hamiltonian of the resonator
def H_r(number_photons, w01, Delta):
    return((w01 + Delta)*number_photons)

# Function that outputs the coupling constant between k,n to k-1, n+1
def geff(g, qubit_state, photon_state, eta, w01):
    just_g = g*(1 + (eta/(2*w01)*qubit_state))
    return(just_g * np.sqrt(qubit_state + 1)*np.sqrt(photon_state))

# Computes the energy eigenstate for RWA strip i.e. Hilbert subspace with constant excitation number
def compute(poss_q, poss_r, N_q, N_r, w, delta, eta, g):

    H = np.zeros((poss_q.size, poss_q.size))
    
    for i in range(poss_q.size) :
        H[i,i] = H_q(poss_q[i], eta, w) + H_r(poss_r[i], w, delta)
        if i < poss_q.size - 1 :
            H[i,i+1] = geff(g, poss_q[i], poss_r[i], eta, w)
            H[i+1,i] = geff(g, poss_q[i], poss_r[i], eta, w)
    
    energy = sorted(np.linalg.eig(H)[0])
    return energy

In [ ]:
# Generates a matrix containing eigenenergy of |k,n> at the (k,n) position
def energyMatrix(N_q, N_r, w, delta, eta, g) :
    emat = np.zeros((N_q, N_r))
    for i in np.arange(0, N_q + N_r -2):
        poss_q = np.flip(np.arange(0,np.min([i+1,N_q])))
        poss_r = np.arange(np.max([0,i+1-(N_q)]),i+1)
        energy = compute(poss_q, poss_r, N_q, N_r, w, delta, eta, g)
        for j in range(poss_q.size):
            if poss_q[j] < N_q and poss_r[j] < N_r:
                emat[poss_q[j],poss_r[j]] = np.real(energy[j])
    return emat

In [ ]:
# Generates two arrays 
# First array is of eigen energies starting at |k,0> to |k,N_r>
# Second array is of excitation number starting at k/n_c to (k + N_r)/n_c
def stripCalc(emat, k, nc, w, delta):
    wbar = emat[k,:] - np.arange(k, emat.shape[1]+k)*(w+delta)
    nphoton = np.arange(k, emat.shape[1]+k)/nc
    return wbar, nphoton

In [ ]:
# Generates two arrays : the purpose is to create arrays with the same start 
# and end of the second array irrespective of transmon state.
## First array is of eigen energies starting at |k,N_q - k> to |k, N_r - k >
## Second array is of excitation number starting at N_q/n_c to N_r/n_c
def standardStrip(emat, k, nc, w, delta) :
    y1, x1 = stripCalc(emat,k,nc,w,delta)
    dellist = np.concatenate((np.arange(k,N_q-1),np.arange(N_r-k,N_r)))
    y1 = np.delete(y1,dellist)
    x1 = np.delete(x1,dellist)
    return y1, x1

In [ ]:
# Returns a line for input of two points and a minimum y value. Given minimum y value is 0
def generateLine(p1x, p1y, p2x, p2y, ymax):
    slope = (p2y - p1y) / (p2x - p1y)
    print("Actual Slope = " + str(slope))
    c = (p1y - slope*p1x)
    x0 = (ymax-c)/slope
    xline = np.linspace(0,x0,50)
    yline = np.linspace(c,ymax,50)
    return xline,yline

In [ ]:
# Values of parameter
# which = 0 Paper Sank et al.
# which = 1 IBM : Armonk
N_q = 20
which = 1
nc = [60,76200][which]
N_r = 20*nc
w = [5.4, 4.972][which]
delta = [1.38, 2.021][which]
eta = [-0.221,-0.348][which]
g = [0.087,0.00366][which]
emat = energyMatrix(N_q, N_r, w, delta, eta, g)
print("nc = " + str((delta/g)**2/4 ))

In [ ]:
# Supplementary Fig 1
print("Slope from - 2g^2/Delta = " + str(-2*g**2/delta))
print("Slope from - eta/(2*nc) = " + str(eta/((2*(delta/g)**2)/4)))
delw = (emat[1,:] - emat[0,:]) - w
nphoton = np.arange(0, N_r)
indx_max = np.argmin(np.abs(delw + 0.5))
plt.plot(nphoton[:indx_max], delw[:indx_max],label = "Numerical")

#line
xline, yline = generateLine(nphoton[0],delw[0],nphoton[1],delw[1], np.min(delw[:indx_max]))
plt.plot(xline,yline, "--", label = "Linear Approximation")
plt.xlabel("Number of photons")
plt.ylabel("AC stark shift (GHz)")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("AC_stark_Shift_IBM.png", dpi = 600, transparent=False)

In [ ]:
# Fig 4.a
%matplotlib inline
clrlist = ["violet", "indigo", "blue", "green","yellow", "orange", "red"]
mylabel = ["$\omega_0$","$\omega_1$","$\omega_2$","$\omega_3$","$\omega_4$","$\omega_5$","$\omega_6$"]
intsct = [[0,5],[0,4]]
thrshld = 0.01
fig, ax1 = plt.subplots()
for i in range(7) :
    y,x = stripCalc(emat,i,nc,w, delta)
    ax1.plot(x,y, label = mylabel[i],color=clrlist[i], zorder=1)
    ax1.plot(x,y + 2*(w+delta), "--",color=clrlist[i], zorder=2)
for i in range(len(intsct)):
    y1, x1 = stripCalc(emat,intsct[i][0],nc,w,delta)
    y2, x2 = stripCalc(emat,intsct[i][1],nc,w,delta)
    y2 = y2 + 2*(w+delta)
    indx = np.where(np.abs(y2-y1) < thrshld)
    ax1.scatter(x1[indx[0][0]], y1[indx[0][0]], color = "black", zorder=3)
    
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.set_xlabel("$n/n_c$")
ax1.set_ylabel("$\omega_k/2\pi$ (in GHz)")
plt.rcParams.update({
    "savefig.facecolor": (1.0, 1.0, 1.0, 1),  # blue  with alpha = 20%
})
fig.tight_layout()
fig.savefig("Fig4a_IBM.png", dpi = 600, transparent=False)

In [ ]:
# Fig 4.b
%matplotlib inline

N_q = 9
nc = 76230
N_r = 5*nc
which = 1
w = [5.4, 4.972][which]
delta = [1.38, 2.021][which]
eta = [-0.221,-0.348][which]
g = [0.087,0.00366][which]

w_r = w + delta
print("The resonator frequency is " + str(w_r) + " GHz")
w = [np.linspace(4.95, 5.6, 12)]
delta = [(w_r - w[0])]
intsct = [[0,5]]
shiftval = [2]


for j in range(len(intsct)):
    intsctX = []
    intsctY = []
    for i in range(w[j].size) :
        emat = energyMatrix(N_q, N_r, w[j][i], delta[j][i], eta, g)
        y1,x1 = standardStrip(emat,intsct[j][0],nc,w[j][i], delta[j][i])
        y2,x2 = standardStrip(emat,intsct[j][1],nc,w[j][i], delta[j][i])
        shift = shiftval[j]*(w[j][i] + delta[j][i])
        indx = np.argmin(np.abs(y1-(y2 + shift)))
        intsctX.append(x1[indx])
        intsctY.append(y1[indx])
    plotx = w[j]
    ploty = np.array(intsctX)
    plt.plot(plotx, ploty, label = str(j))
plt.xlabel("w")
plt.ylabel("n/nc")
plt.ylim((0,12))
plt.savefig("Fig4b_IBM.png",dpi = 600)